In [1]:
import pandas as pd 
import numpy as np
from pathlib import Path

In [1]:
src_files = Path('src_csv').glob('*.csv')
output_path = Path('src_forms4/csv')
pool_of_filds = pd.read_csv(Path('src_forms3/raw/forms_raw6.csv'), dtype=str) #исходый файл предварительно орботан вручную

NameError: name 'Path' is not defined

In [3]:
def create_unit_key(x):
    x = x.dropna()
    if 'label' and 'value' in x:
        return ','.join([x['label'], x['value'], 'Нет'])
    elif 'label' and 'options' in x:
        x['options'] = x['options'].split('\\r\\n')
        for y in range(len(x['options'])):
            x['options'][y] = x['options'][y].split('=')[0]
        x['options'] = ','.join(x['options'])
        return ','.join([x['label'], x['options']])
    elif 'label' in x:
        return x['label']
    else :
        return ','.join(x.dropna().astype(str))

In [4]:
for i in src_files:
    result = pd.DataFrame()
    result['unitkey'] = pd.read_csv(i, dtype=str).dropna(how='all').apply(lambda x: create_unit_key(x) ,axis=1)
    result = pd.merge(result, pool_of_filds, how='left', on='unitkey')
    result = result[['name', 'type', 'label', 'value', 'options']]
    csv_file_stem = result['name'][result['type']=='divider'].iloc[0] or False
    file_name = output_path / (csv_file_stem + '.csv') if csv_file_stem else output_path / i.name
    result.to_csv(file_name, index=False)